# Topic Modelling With Constellate using LDA and NMF

Paul King

View in GitHub: [Topic Modelling with Constellate](https://github.com/paulkdata/topic_modelling_with_constellate)

Or, [Launch in Constellate](https://constellate.org/notebook/own/?repo=https%3A%2F%2Fgithub.com%2Fpaulkdata%2Ftopic_modelling_with_constellate&urlpath=tree%2Ftopic_modelling_with_constellate)

## Overview
This Jupyter notebook-based tutorial describes how to perform topic modelling with the LDA and NMF algorithms on a corpus, and how to quantitatively and qualitatively evaluate them. This tutorial can be run through the Constellate website, so only an internet connection and a JSTOR account linked to your institution is required. If you have a Jupyter notebook/Python environment, you may run this tutorial on your local host, however a JSTOR account is required to build your own dataset. Make sure to download the Constellate Client library if you are running this tutorial locally: [Consellate Client](https://constellate.org/docs/constellate-client/)

## Target Audience
This tutorial is suitable both for independent learners interested in text analytics and digital humanities, as well as in a classroom or workshop with an instructor. Learners should have some base understanding of using Python and Jupyter notebooks, as well as knowledge of basic text analytics and interest in learning more. If you are not familiar with Python or text analytics such as tokenizing text and using regular expressions, Constellate has a wide variety of tutorials available here: [Constellate Tutorials](https://constellate.org/tutorials)

## Learning Outcomes
Upon completion of this tutorial, learners will be able to:
- Download and convert text data into a format understandable by topic models
- Run LDA and NMF topic models using the Gensim library
- Quantitatively evaluate topic models on computational time and coherence score
- Qualitatively evaluate topic models based on resulting topics

## Topic Modelling with Constellate
Topic modelling is a statistical text mining technique. Topic modelling is used to extract "topics", representative sets of words, from a corpus of texts. The most common algorithm for topic modelling is called Latent Dirichlet Allocation, or LDA. You can use LDA in Constellate, however, because Constellate uses cloud computing, topic modelling with LDA can be very slow. In such cases, it can be quicker to use an algorithm called Non-Negative Matrix Factorization, or NMF. In this tutorial, we will import a corpus, pre-process unigram tokens, and perform topic modelling with LDA and NMF and compare them by time, coherence score, and resultant topics.

Before doing this tutorial, I recommend that you are familiar with the basics of Constellate and using Python in Jupyter notebooks. It will also help to be familiar with basic text analytics including pre-processing and tokenizing texts, however this code will be supplied in the tutorial. Constellate has many additional tutorials on their website for text analytics.

Both LDA and NMF are available through the Gensim library.

Note that some code has been adapted from Nathan Kelber and Ted Lawless's Topic Modelling tutorial: [Latent Dirichlet Allocation (LDA) Topic Modeling
](https://github.com/ithaka/tdm-notebooks/blob/master/topic-modeling.ipynb)

## Getting Started
To begin, we need to import the libraries and download the dataset. Besides some standard libraries, Gensim is a Python library for topic modelling and other natural language processing techniques, and has support for both LDA and NMF, as well as calculating coherence scores.

You will also need to download the dataset from Constellate. If you have a dataset that you'd like to perform topic modelling on, you can replace the dataset ID provided with your own dataset.

If you are running this tutorial in Constellate, make sure that you have a JSTOR account and link your account to your institution: [Constellate Log-in](https://constellate.org/docs/log-in)

If you are running this tutorial through Constellate, please note that Constellate uses an older version of the Gensim library that does not include NMF. To update to the latest version of Gensim, you can run the following code block and then restart the kernel before proceeding. More information is available here: [How to install packages in Constellate](https://constellate.org/docs/install-a-package-in-constellate)

In [ ]:
# Run this code first to update the Gensim library and then restart the kernel (type 0 twice quickly)
!pip install gensim -U

In [1]:
# Import libraries and dataset
import constellate
import gensim
import re
import time
import pandas as pd

Constellate: use and download of datasets is covered by the Terms & Conditions of Use: https://constellate.org/terms-and-conditions/
INFO:numexpr.utils:NumExpr defaulting to 6 threads.


In [2]:
# Import the dataset

# If you have created a dataset with the Constellate Dataset Builder, you can input it here
# Otherwise, you may use the dataset supplied with this tutorial
# Default is "digital scholarship" from JSTOR or Portico limited to document type(s) article from 1900 - 2023 - (2,406 total documents)

dataset_id = "d96db0ac-fb15-d85a-2e88-fe9025c7cf5c" # Replace with your own dataset ID
dataset_file = constellate.get_dataset(dataset_id)

"digital scholarship" from JSTOR or Portico limited to document type(s) article from 1900 - 2023. 2406 documents.
INFO:root:File C:\Users\Paul K\data\d96db0ac-fb15-d85a-2e88-fe9025c7cf5c-sampled-jsonl.jsonl.gz exists. Not re-downloading.


## Stoplist
Before we proceed with pre-processing tokens, we will want a stoplist. A stoplist is a list of words we will omit from our topic model, because they are too common or provide no semantic value. The NLTK library (Natural Language Toolkit) provides a basic stoplist of very common English words. However, you will likely find words particular to your dataset that show up in most documents, and thus will reduce your topic model's effectiveness. For instance, in our example dataset, because our documents are about Digital Scholarship, we find that most documents will include the word "information", so we want to remove it. Topic modelling is an iterative process; it is normal to run your topic model, look at the resultant topics, then go back and extend your stoplist with words that you do not want in your topic model. Another way to extend your stoplist is to calculate the frequency of the most common tokens and find the most common terms.

In [3]:
#Import stopword list
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
new_stoplist = [] # You can extend your stoplist with additional words
# Example:
# new_stoplist = ["libraries","library","information"]
stop_words.extend(new_stoplist)

## Pre-processing tokens
Constellate datasets supply unigram tokens directly, which are essentially single word units that can be understood by topic models. This is drastically simpler than, say, scraping websites for text data, which would then need to be converted into unigrams. However, these tokens can still contain punctuation and numbers which do not contain semantic information and will prevent the topic model from reading them properly. Tokens can also contain upper-case characters which need to be converted to lower-case. We will define a function in the next code block that will be used to pre-process unigram tokens as they are extracted from the dataset. We use a Regular Expression to remove all non-alphabetical characters.

The script below also includes a lemmatizer from NLTK. The lemmatizer converts inflected forms of words into their lemma, or base forms. We would lemmatize "worse" to "bad" and "running" to "run". This way, unigrams with the same lemma are treated the same way in the topic model. If you have a very large dataset that is slow to pre-process, you could consider removing the lemmatizer, however this will decrease the accuracy of the topic model.

In [4]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def process_token(token):
    token = token.lower()
    token = re.sub('[^a-z]+', '', token)
    token = wordnet_lemmatizer.lemmatize(token) # The lemmatizer is very slow, so you may comment this line out with # to speed up the next step
    if token in stop_words:
        return
    if len(token) < 4:
        return
    if not(token.isalpha()):
        return
    return token

With our process_token function defined, we can proceed with extracting and pre-processing unigram tokens from documents. We retrieve documents with the constellate.dataset_reader function, then iterate through each unigram in each document, creating a list of pre-processed unigram tokens for each document.

In [5]:
# Import unigram counts from dataset
# Datasets in Constellate already have extracted unigrams, which simplifies our process

documents = []
for document in constellate.dataset_reader(dataset_file):
    processed_document = []
    unigrams = document.get("unigramCount", {})
    for gram, count in unigrams.items():
        clean_gram = process_token(gram)
        if clean_gram is None:
            continue
        processed_document += [clean_gram] * count # Add the unigram as many times as it was counted
    if len(processed_document) > 0:
        documents.append(processed_document)

In [ ]:
documents[1] # Run this code block to see what a pre-processed document looks like

The next two steps will use built-in gensim functions. The first step maps every word to an integer ID in a "dictionary". The second step converts each document into a "bag of words", a list of each integer ID paired with a count of tokens for that particular document. The result is a document-term matrix, which is necessary for topic modelling. Explaining these functions is beyond the scope of this tutorial, but you can consult the [Gensim API reference](https://radimrehurek.com/gensim/apiref.html) if you would like to learn more.

In [6]:
# Create dictionary
dictionary = gensim.corpora.Dictionary(documents)

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>
INFO:gensim.corpora.dictionary:built Dictionary<274592 unique tokens: ['ability', 'able', 'academic', 'access', 'accessible']...> from 1500 documents (total 9300191 corpus positions)
INFO:gensim.utils:Dictionary lifecycle event {'msg': "built Dictionary<274592 unique tokens: ['ability', 'able', 'academic', 'access', 'accessible']...> from 1500 documents (total 9300191 corpus positions)", 'datetime': '2023-04-09T20:18:19.914746', 'gensim': '4.2.0', 'python': '3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}


In [7]:
bow_corpus = [dictionary.doc2bow(doc) for doc in documents]

We will also create an empty dataframe to store our results. We want to know the model used (LDA or NMF), and the number of topics and passes that we passed as input, as well as the resultant coherence score and computation time. We will retrieve this dataframe later for quantitative evaluation.

In [8]:
# Quantitative results dataset
results_df = pd.DataFrame(columns = ["Model","Coherence Score","Computation Time","Number of Topics","Passes"])
results_df

,Model,Coherence Score,Computation Time,Number of Topics,Passes


## Coherence scores
If we want to be confident in our topic models, we would like a way to automatically evaluate whether our topics are coherent, and actually represent what a human would consider a coherent topic; that is to say, the words in the topic are relevant and appear to be related. Topic coherence is a measure of semantic similarity between terms in a topic; higher (positive) topic coherence correlates with greater semantic coherence in human evaluation. Gensim provides a function for calculating the coherence score of a topic model. We will integrate this function into a new function, which we will include in our topic model code to calculate and print the topic coherence score.

Perplexity is another common metric used to evaluate topic models, used to show how successful a topic model can predict held-out documents from the same corpus. However, topic models that perform well with regards to perplexity do not necessarily have high coherence, by human estimation, nor by calculation (Chang et al., 2009). As well, calculating perplexity is challenging (Wallach et al., 2009), and Gensim supplies "per-word likelihood bound" for LDA, a method for estimating perplexity, however they do not provide any function for estimating perplexity for NMF. For these reasons it makes sense to omit perplexity from our evaluation.

In [9]:
# Coherence function
from gensim.models import CoherenceModel

def coherenceScore(model):
    # Compute the coherence score using UMass
    # u_mass is measured from -14 to 14, higher is better
    coherence_model = CoherenceModel(
        model=model,
        corpus=bow_corpus,
        dictionary=dictionary, 
        coherence='u_mass'
    )

    # Compute Coherence Score using UMass
    coherence_score = coherence_model.get_coherence()
    print('\nCoherence Score: ', coherence_score)
    return coherence_score

## The topic models
The next two code blocks will produce the topic models. For both LDA and NMF, we need to supply the document-term matrix, the dictionary, the number of topics and the number of passes through the corpus. In theory, more passes will result in more coherent topics.

We use the Time Python library to calculate the time it takes for the model to run in seconds. Besides running the model, we want to collect the coherence score and computation time and add them to our Results DataFrame for future evaluation.

Optimizing the ideal number of topics and passes is a nontrivial task. More passes will also mean longer computation time. For those with greater comfort in Python, it is possible to iterate over a selection of numbers of topics and passes, and plot the resulting coherence scores.

In [ ]:
# We would like to keep our number of topics and passes constant, so we will set the same values for both algorithms
num_topics = 7 # Change this to alter the number of topics
passes = 3 # Change this to alter the number of passes

In [10]:
# LDA
# Train the LDA model
start = time.time()

LDAmodel = gensim.models.LdaModel(
    corpus=bow_corpus,
    id2word=dictionary,
    num_topics=num_topics,
    passes=passes
)

end = time.time()

coherence = coherenceScore(LDAmodel)
print(end-start)
results_df.loc[len(results_df.index)] = ["LDA",coherence,end-start,num_topics,passes]

INFO:gensim.models.ldamodel:using symmetric alpha at 0.14285714285714285
INFO:gensim.models.ldamodel:using symmetric eta at 0.14285714285714285
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online (multi-pass) LDA training, 7 topics, 3 passes over the supplied corpus of 1500 documents, updating model once every 1500 documents, evaluating perplexity every 1500 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamodel:-13.205 per-word bound, 9439.8 perplexity estimate based on a held-out corpus of 1500 documents with 9300191 words
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #1500/1500
INFO:gensim.models.ldamodel:topic #4 (0.143): 0.007*"library" + 0.006*"information" + 0.006*"research" + 0.006*"university" + 0.005*"digital" + 0.005*"journal" + 0.005*"data" + 0.004*"study" + 0.003*"work" + 0.003*"student"
INFO:gensim.models.ldamodel:topic #5 (0.143): 0.009*"library" + 0.009*"informati

In [11]:
# NMF
# Train the NMF model
start = time.time()

NMFmodel = gensim.models.nmf.Nmf(
    corpus=bow_corpus,
    id2word=dictionary,
    num_topics=num_topics,
    passes=passes
)

end = time.time()

coherence = coherenceScore(NMFmodel)
print(end-start)
results_df.loc[len(results_df.index)] = ["NMF",coherence,end-start,num_topics,passes]

INFO:gensim.models.nmf:running NMF training, 7 topics, 3 passes over the supplied corpus of 1500 documents, evaluating L2 norm every 1500 documents
INFO:gensim.models.nmf:PROGRESS: pass 0, at document #1500/1500
INFO:gensim.models.nmf:L2 norm: 27637.779991499665
INFO:gensim.models.nmf:topic #0 (0.000): 0.000*"alonainy" + 0.000*"januaryjuly" + 0.000*"callon" + 0.000*"libertas" + 0.000*"rotten" + 0.000*"ared" + 0.000*"chemosphere" + 0.000*"menstruation" + 0.000*"sessanta" + 0.000*"embeddings"
INFO:gensim.models.nmf:topic #1 (0.000): 0.000*"softbook" + 0.000*"etezadiamoli" + 0.000*"canonical" + 0.000*"httpthatcamporg" + 0.000*"wwwmuseumsandthewebcommwpapersex" + 0.000*"indicted" + 0.000*"hrlindgrenpipl" + 0.000*"archaeoastronomers" + 0.000*"unavailable" + 0.000*"nowviskieorgnwherecreditisdue"
INFO:gensim.models.nmf:topic #4 (0.000): 0.000*"ispirato" + 0.000*"tawiah" + 0.000*"domaingeneral" + 0.000*"onlyeasily" + 0.000*"advocacy" + 0.000*"courier" + 0.000*"universitaa" + 0.000*"sioning" + 

## Quantitative Analysis
Having run both LDA and NMF models, we would like to see the results. In general, LDA models will have higher coherence scores, whereas NMF models will have faster computation times (Egger and Yu, 2022). However, there is no one-size-fits-all option for topic modelling, and optimizing your topic model will be dependent on your particular dataset. More importantly, we must consider these quantitative results in the context of the resulant topics, which is covered in the next step.

In [12]:
# Execute this code block to compare results of all topic models performed
results_df

,Model,Coherence Score,Computation Time,Number of Topics,Passes
0,LDA,-0.342030,58.499248,7,3
1,NMF,-0.847516,30.322424,7,3


## Qualitative Analysis
Besides coherence scores and computation times, we will also need to qualitatively evaluate the resultant topics of our models. Again, this is This qualitative analysis is best suited for a domain expert familiar with the corpus. We are looking for topics that have an internal semantic coherence. However, we would also like to see that our topics are suitably dissimilar, without significant overlap. Topic modelling is an iterative proces. If you find that there is too much overlap in your topics, consider adding additional terms to your stoplist, or increasing the number of passes of your topic models. If you are satisfied with your topics, consider finding the documents which best match your topics and reading them to better learn what your topics represent.

In [35]:
# Print topics of two last models
# need to adapt this code to match the rest of the tutorial
# Retrieve the topics for k = 5
pd.set_option('display.max_colwidth', None)

lda_topics = []
for i in range(num_topics):
    topic = LDAmodel.show_topic(i,10)
    lda_topics.append([x for x,y in topic])
lda_topics_df = pd.DataFrame(lda_topics,index=range(num_topics))
lda_topics_df['Topics'] = lda_topics_df.values.tolist()
lda_topics_df = pd.DataFrame(lda_topics_df['Topics'],index=range(num_topics))
print(lda_topics_df)
nmf_topics = []
for i in range(num_topics):
    topic = NMFmodel.show_topic(i,10)
    nmf_topics.append([x for x,y in topic])
nmf_topics_df = pd.DataFrame(nmf_topics,index=range(num_topics))
nmf_topics_df['Topics'] = nmf_topics_df.values.tolist()
nmf_topics_df = pd.DataFrame(nmf_topics_df['Topics'],index=range(num_topics))
print(nmf_topics_df)


                                                                                               Topics
0                    [digital, history, work, also, project, study, university, book, research, text]
1                [university, library, study, digital, research, information, also, text, work, data]
2      [research, journal, data, study, library, information, university, language, digital, patient]
3             [univ, study, state, student, program, library, research, american, literature, social]
4  [university, research, digital, journal, knowledge, information, literature, study, univ, library]
5          [library, information, research, data, study, university, student, journal, digital, user]
6         [library, research, digital, student, study, service, university, journal, also, available]
                                                                                              Topics
0        [patient, pediatric, cancer, child, hospital, oncology, treatment, year, t

## Discussion
Text analytic algorithms such as LDA and NMF are often evaluated by quantitative metrics such as perplexity and coherence. However, in the age of Large Language Models such as OpenAI's GPT-4, there is also a need to evaluate these algorithms by time and energy usage (Strubell et al., 2019). Topics must also be evaluated qualitatively by a domain expert, as topic models with poorly tuned parameters can have flaws that are obvious to experts but not novices (Mimno et al., 2011). It is my hope that this tutorial has inspired you to think about these algorithms with respect to computational costs and qualitative analysis.

There are other algorithms that can be used for topic modelling, including Biterm Topic Modelling and Correlation Explanation (or CorEx). CorEx differs from other topic modelling algorithms in that it permits the use of "semi-supervised modelling", allowing scholars to provide anchor terms for topics.

## Citations
Chang, J., Gerrish, S., Wang, C., Boyd-graber, J., & Blei, D. (2009). Reading Tea Leaves: How Humans Interpret Topic Models. Advances in Neural Information Processing Systems, 22. https://papers.nips.cc/paper/2009/hash/f92586a25bb3145facd64ab20fd554ff-Abstract.html

Egger, R., & Yu, J. (2022). A Topic Modeling Comparison Between LDA, NMF, Top2Vec, and BERTopic to Demystify Twitter Posts. Frontiers in Sociology, 7. https://www.frontiersin.org/articles/10.3389/fsoc.2022.886498

Mimno, D., Wallach, H., Talley, E., Leenders, M., & McCallum, A. (2011). Optimizing Semantic Coherence in Topic Models. Proceedings of the 2011 Conference on Empirical Methods in Natural Language Processing, 262–272.

Strubell, E., Ganesh, A., & McCallum, A. (2019). Energy and Policy Considerations for Deep Learning in NLP. ArXiv:1906.02243 [Cs]. http://arxiv.org/abs/1906.02243

Wallach, H. M., Murray, I., Salakhutdinov, R., & Mimno, D. (2009). Evaluation methods for topic models. Proceedings of the 26th Annual International Conference on Machine Learning, 1105–1112. https://doi.org/10.1145/1553374.1553515

## Resources and Recommended Reading
- Constellate tutorials

- BTM

- CorEx